In [1]:
import pandas as pd 
import numpy as np

In [2]:
train = pd.read_csv('data/v2/training_v2.csv')
test = pd.read_csv('data/v2/test_v2.csv')

In [8]:
train.Type.fillna(value="Nope", inplace=True)
test.Type.fillna(value="Nope", inplace=True)

In [9]:
train.head()

,application_date,segment,case_count,day,year,month,Holiday,Type,cos_day,sin_day,cos_mon,sin_mon
0,2017-04-01,1,40.0,1,2017,4,NaN,Nope,0.978148,0.207912,-0.5,0.866025
1,2017-04-03,1,5.0,3,2017,4,NaN,Nope,0.809017,0.587785,-0.5,0.866025
2,2017-04-04,1,4.0,4,2017,4,Rama Navami,G,0.669131,0.743145,-0.5,0.866025
3,2017-04-05,1,113.0,5,2017,4,NaN,Nope,0.500000,0.866025,-0.5,0.866025
4,2017-04-07,1,76.0,7,2017,4,NaN,Nope,0.104528,0.994522,-0.5,0.866025


In [11]:
test.head()

,id,application_date,segment,day,year,month,Holiday,Type,cos_mon,sin_mon,cos_day,sin_day
0,1,2019-07-06,1,6,2019,7,NaN,Nope,-0.866025,-0.5,0.347305,0.937752
1,2,2019-07-07,1,7,2019,7,NaN,Nope,-0.866025,-0.5,0.151428,0.988468
2,3,2019-07-08,1,8,2019,7,NaN,Nope,-0.866025,-0.5,-0.050649,0.998717
3,4,2019-07-09,1,9,2019,7,NaN,Nope,-0.866025,-0.5,-0.250653,0.968077
4,5,2019-07-10,1,10,2019,7,NaN,Nope,-0.866025,-0.5,-0.440394,0.897805


# Encoding Type of Holiday

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
on = OneHotEncoder()

In [18]:
type_mat = on.fit_transform(train['Type'].values.reshape(-1,1))

In [29]:
mat = type_mat.todense()

In [30]:
mat.shape

(82398, 7)

In [31]:
mat = np.append(mat, train['segment'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['cos_day'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['sin_day'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['cos_mon'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['sin_mon'].values.reshape(-1, 1), axis=1)

In [32]:
mat.shape

(82398, 12)

In [39]:
test.head()

,id,application_date,segment,day,year,month,Holiday,Type,cos_mon,sin_mon,cos_day,sin_day
0,1,2019-07-06,1,6,2019,7,NaN,Nope,-0.866025,-0.5,0.347305,0.937752
1,2,2019-07-07,1,7,2019,7,NaN,Nope,-0.866025,-0.5,0.151428,0.988468
2,3,2019-07-08,1,8,2019,7,NaN,Nope,-0.866025,-0.5,-0.050649,0.998717
3,4,2019-07-09,1,9,2019,7,NaN,Nope,-0.866025,-0.5,-0.250653,0.968077
4,5,2019-07-10,1,10,2019,7,NaN,Nope,-0.866025,-0.5,-0.440394,0.897805


In [45]:
mat_test = on.transform(test['Type'].values.reshape(-1,1))

In [46]:
mat_test = mat_test.todense()

In [47]:
mat_test = np.append(mat_test, test['segment'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['cos_day'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['sin_day'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['cos_mon'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['sin_mon'].values.reshape(-1, 1), axis=1)

In [48]:
mat_test.shape

(182, 12)

In [81]:
from sklearn.linear_model import LinearRegression, HuberRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

In [107]:
X_train, X_test, y_train, y_test = train_test_split(mat, train['case_count'], test_size=0.33, random_state=42)

In [108]:
lr = LinearRegression()
hu = HuberRegressor()
sgd = SGDRegressor()

In [109]:
rf = RandomForestRegressor(n_estimators=100)
et = ExtraTreesRegressor(n_estimators=100)

In [110]:
rf.fit(X_train, y_train)
et.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [52]:
lr.fit(X_train, y_train)
hu.fit(X_train, y_train)
sgd.fit(X_train, y_train)

C:\Users\EW\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [75]:
lr_pred = np.round(abs(lr.predict(X_test)))
hu_pred = np.round(abs(hu.predict(X_test)))
sgd_pred = np.round(abs(sgd.predict(X_test)))

In [113]:
rf_pred = np.round(abs(rf.predict(X_test)))
et_pred = np.round(abs(et.predict(X_test)))

In [112]:
mean_absolute_error(y_test, lr_pred) * 100/182, mean_absolute_error(y_test, hu_pred) * 100/182, mean_absolute_error(y_test, sgd_pred) * 100/182

(122.75519787655709, 89.52239103938133, 124.95192913882234)

In [114]:
mean_absolute_error(y_test, rf_pred) * 100/182, mean_absolute_error(y_test, et_pred) * 100/182

(101.79159432800209, 101.88696820978376)

124.95192913882234

In [115]:
test_pred = np.round(rf.predict(mat_test))

In [116]:
test['case_count'] = test_pred.reshape(-1,1)

In [117]:
test[['id','application_date','segment','case_count']].to_csv('pred_baseline_rf.csv', index=False)

# LGM

In [85]:
median_val = np.median(train['case_count'].values)

In [89]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmsle',
    'max_depth': 6, 
    'learning_rate': 0.1,
    'verbose': 0}
#     'early_stopping_round': 20}
n_estimators = 100

In [124]:
n_iters = 5
preds_buf = []
err_buf = []
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(np.array(mat), train['case_count'].values.reshape(-1,1).ravel(), test_size=0.1, random_state=i)
    d_train = lgb.Dataset(x_train, label=y_train)
    d_valid = lgb.Dataset(x_valid, label=y_valid)
    watchlist = [d_valid]

    model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

    preds = model.predict(x_valid)
#     preds = np.exp(preds)
    preds[preds < 0] = median_val
    err = mean_absolute_error(y_valid, preds) * 100/len(y_valid)
    err_buf.append(err)
    print('MAE% = ' + str(err))
    
    preds = model.predict(np.array(mat_test))
#     preds = np.exp(preds)
    preds[preds < 0] = median_val
    preds_buf.append(preds)

print('Mean MAE% = ' + str(np.mean(err_buf)) + ' +/- ' + str(np.std(err_buf)))
# Average predictions
preds = np.round(np.mean(preds_buf, axis=0))

MAE% = 2.133783523513568
MAE% = 2.226129904020018
MAE% = 2.161169430293224
MAE% = 2.1387945927443752
MAE% = 2.3745359280677167
Mean MAE% = 2.2068826757277806 +/- 0.09005247661674118


In [96]:
len(train['case_count'].values.reshape(-1,1).ravel())

82398

In [121]:
mat_test

matrix([[ 0.        ,  0.        ,  0.        , ...,  0.93775213,
         -0.8660254 , -0.5       ],
        [ 0.        ,  0.        ,  0.        , ...,  0.98846832,
         -0.8660254 , -0.5       ],
        [ 0.        ,  0.        ,  0.        , ...,  0.99871651,
         -0.8660254 , -0.5       ],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.96807712,
          0.5       , -0.8660254 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.99871651,
          0.5       , -0.8660254 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.98846832,
          0.5       , -0.8660254 ]])

In [128]:
test['case_count_lgb'] = preds

In [134]:
test = test.drop_duplicates(subset=['id'])

In [135]:
test[['id','application_date','segment','case_count_lgb']].to_csv('pred_baseline_lgb.csv', index=False)